# Treinamento e validação de Modelos - Dataset Combinado
Treinamento de um modelo usando como base um dataset combinado, constituído dos dois datasets utilizados ao longo do projeto: 
- Fake.br-Corpus: https://github.com/roneysco/Fake.br-Corpus
- FakeRecogna: https://github.com/Gabriel-Lino-Garcia/FakeRecogna 

Espera-se que um modelo treinado a partir de ambos datasets possua um desempenho melhor em uma maior variedade de notícias.

In [1]:
# Se necessário
%pip install wordcloud

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import sagemaker
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import classification_report
from scipy.sparse import csr_matrix, hstack
from python_scripts.save_load import load_df_from_bucket, save_df_to_s3_bucket, save_to_s3_bucket_as_libsvm, BUCKET_MODEL
from python_scripts.modelling import create_train_validation_test_sets, setup_model, make_prediction

## Carregamento de dados
O dataset combinado já foi criado e salvo no notebook `preprocessing.ipynb`. Aqui, ele é apenas carregado novamente a partir do S3.

In [3]:
model_df = load_df_from_bucket('dados_processados_combinados.csv', tipo='processado')
model_df.head()

,fake,lemmas_str
0,0,divisão STF meio partidário independente ficar...
1,1,general mandar recado STF abaixar calça congre...
2,1,nordeste acordar Lula PT enxotar chegar bandei...
3,0,dois relatório Polícia Federal análise materia...
4,1,Coreia Norte declarar status QUASE-GUERRA mobi...


## Modelo 1: TF-IDF baseado no texto da notícia
Vetorização TF-IDF é aplicada somente à coluna de texto da notícia. Demais colunas de dados não são consideradas.

In [4]:
# Tamanhos de teste e validação garantem que o arquivo de teste seja < 5 MB, facilita implementação
train_1, test_1, validate_1 = create_train_validation_test_sets(model_df, 
                                                                stratify_col='fake',
                                                                test_size=0.1, random_state=42,
                                                                validation_size=0.55)

### Processamento adicional

Um vetorizador TFIDF é utilizado para converter os dados textuais em colunas do DataFrame.

In [5]:
# Criar vetorizador TFIDF e ajustar aos dados de treinamento
tfidf = TfidfVectorizer(lowercase=False, ngram_range = (1,2))
tfidf.fit(train_1['lemmas_str'])

TfidfVectorizer(lowercase=False, ngram_range=(1, 2))

In [6]:
def create_x_y_1(base_df, tfidf, target_col='fake', lemma_col = 'lemmas_str'):
    tfidf_res = tfidf.transform(base_df[lemma_col])
    return tfidf_res, base_df[target_col]

x_train_1, y_train_1 = create_x_y_1(train_1, tfidf)
x_validate_1, y_validate_1 = create_x_y_1(validate_1, tfidf)
x_test_1, y_test_1 = create_x_y_1(test_1, tfidf)

### Upload de dados para o S3

In [7]:
file_name_tuples = [(x_train_1, y_train_1, 'train'), 
                   (x_test_1, y_test_1, 'test'), 
                   (x_validate_1, y_validate_1, 'validate')]

for x, y, prefix in file_name_tuples:
    save_to_s3_bucket_as_libsvm(x, y, prefix=prefix, filename='model_1_combo.libsvm', tipo='modelo')

/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/home/ec2-user/anaconda3/envs/amazonei_tensorflow2_p36/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
/home/ec2-user/anaconda3/envs/amazon

### Treinar modelo

In [8]:
xgb_model_1, data_channels_1 = setup_model(base_image='xgboost', model_name='model_1_combo', instance_count=4, 
                                           instance_type='ml.m4.xlarge')
xgb_model_1.fit(inputs=data_channels_1)

print('ready for hosting!')

2022-11-15 13:05:00 Starting - Starting the training job...ProfilerReport-1668517500: InProgress
...
2022-11-15 13:05:45 Starting - Preparing the instances for training.........
2022-11-15 13:07:26 Downloading - Downloading input data......
2022-11-15 13:08:26 Training - Downloading the training image.....[2022-11-15 13:09:09.786 ip-10-0-109-253.ec2.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-11-15:13:09:09:INFO] Imported framework sagemaker_xgboost_container.training
[2022-11-15:13:09:09:INFO] Failed to parse hyperparameter eval_metric value auc to Json.
Returning the value itself
[2022-11-15:13:09:09:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2022-11-15:13:09:09:INFO] No GPUs detected (normal if no gpus installed)
[2022-11-15:13:09:09:INFO] Running XGBoost Sagemaker in algorithm mode
[2022-11-15:13:09:09:INFO] files path: /opt/ml/input/data/train
[2022-11-15:13:09:10:INFO] files path: /opt/ml/in

In [9]:
xgb_predictor_1 = xgb_model_1.deploy(initial_instance_count=1,
                                     serializer=sagemaker.serializers.LibSVMSerializer(),
                                     instance_type='ml.m4.xlarge')

--------!

### Métricas do modelo

In [10]:
y_pred_1 = make_prediction(xgb_predictor_1, model_name='model_1_combo')

In [11]:
print(classification_report(y_test_1, y_pred_1))

              precision    recall  f1-score   support

           0       0.94      0.87      0.90       430
           1       0.88      0.94      0.91       429

    accuracy                           0.91       859
   macro avg       0.91      0.91      0.91       859
weighted avg       0.91      0.91      0.91       859



Verifica-se aqui um excelente desempenho mesmo com os dados separados para teste, como foi visto no notebook `model_fakebr.ipynb`para o dataset Fake.br-Corpus. Em seguida, será feita uma segunda validação utilizando este dataset.

### Salvar predição

In [12]:
df_pred_alt = pd.DataFrame({'pred_1_combo': y_pred_1})
df_pred_alt.to_csv(f's3://{BUCKET_MODEL}/test/pred_1_combo.csv', encoding='utf-8', index=False)

### Encerrar modelo

In [13]:
xgb_predictor_1.delete_endpoint(delete_endpoint_config=True) 